In [2]:
import pandas as pd
import numpy as np
import time
import os

t = time.localtime()
timestamp = time.strftime('%y%m%d-%H%M', t)

# read excel
df = pd.read_excel('H://Current_Working_Folder//Reports Analytics//202106//Version 2//202106-t_worc_families-20210810_1309.xlsx')

# assign path and reporitng month
drive = 'H://Current_Working_Folder//Reports Analytics//'
reporting_month = str(df.loc[0,'reporting_month'])

# Filter with column names ending with amt, amount, nbr_month
df_amt = df.filter(regex = 'amt$|amount$|amount$|nbr_month$|nbr_months', axis='columns')

# Describe the filtered dataframe
df_des = df_amt.describe()

# Write the describe to excel
writer = pd.ExcelWriter(drive + reporting_month + '//' + reporting_month + '_' + 'report' + '-' + timestamp + '.xlsx')
#df_des.to_excel(writer, index = False)
df_des.to_excel(writer, sheet_name='family_stat')
writer.save()

In [3]:
#df.columns

In [4]:
df.shape

(1553, 56)

In [5]:
df['subsid_child_care'].value_counts()

No subsidized child care received                                                                                                                 1463
Yes, receives child care funded entirely or in part with Federal funds (e.g., receives TANF, CCDF, SSBG, or other federally funded child care)      90
Name: subsid_child_care, dtype: int64

In [6]:
df['subsid_child_care_amt'].sum()

84438

In [7]:
df['tanf child care_children_covered'].value_counts()

0    1553
Name: tanf child care_children_covered, dtype: int64

In [8]:
df['tanf child care_amount'].sum()

0

In [9]:
cat = df.drop(df_amt, axis='columns')
#cat1 = cat.drop((cat.loc[:,'reporting_month':'stratum']),axis='columns')
cat1 = cat.drop(['id','reporting_month','case_number','county_fips_code','stratum','disposition','worker_id',
                 'case_manager','area','created_by','updated_by','created_at','updated_at','ans_county',
                 'local_office','waiver_grps'], axis='columns')
cat2 = cat1.drop(cat1.filter(regex = '^reason|^other'), axis='columns')

In [10]:
reason_cat = cat1.filter(regex = '^reason', axis='columns')

for c in reason_cat.columns:
    print(reason_cat[c].value_counts())
    print('---')


No     1301
Yes     252
Name: reason for amount of reduction_work requirements sanction, dtype: int64
---
No    1553
Name: reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent, dtype: int64
---
No    1553
Name: reason for amount of reduction_Sanction for Teen Parent not Attending School, dtype: int64
---
No     1456
Yes      97
Name: reason for amount of reduction_Non-Cooperation with Child Support, dtype: int64
---
No     1548
Yes       5
Name: reason for amount of reduction_Failure to Comply with an Individual Responsibility Plan, dtype: int64
---
No     1551
Yes       2
Name: reason for amount of reduction_Other Sanction, dtype: int64
---
0    1553
Name: reason for amount of reduction_Recoupment of Prior Overpayment, dtype: int64
---


In [11]:
#((df.filter(regex = '^reason'))).head(20)

In [12]:
for c in cat2.columns:
    print((cat2[c].value_counts().to_frame('count').rename_axis(c)).reset_index())
    print('---')

   category  count
0       Tea   1475
1  WorkPays     78
---
     zip_code  count
0       71854     60
1       72301     45
2       72209     45
3       72390     41
4       72401     36
..        ...    ...
300     72131      1
301     72460      1
302     72858      1
303     72135      1
304     72137      1

[305 rows x 2 columns]
---
                                      funding_stream  count
0  Funded, in whole or in part, with Federal TANF...   1553
---
                       new_applicant  count
0     Not newly-approved application   1549
1  Yes, a newly-approved application      4
---
   family_size  count
0            2    591
1            1    458
2            3    310
3            4    130
4            5     47
5            6     11
6            7      4
7           10      1
8            8      1
---
                                         family_type  count
0            Family with no work-eligible individual    859
1  Family included only in overall work participa...   

In [13]:
worc_fam = df
fam_type = worc_fam.groupby(['category','family_type']).agg({'case_number' : 'nunique'}).rename({'case_number': 'case_count'}, axis='columns')#.reset_index()
fam_type

case_count
category family_type                                                   
Tea      Family included only in overall work participat...         591
         Family with no work-eligible individual                    859
         Two-Parent Family included in both the overall ...          25
WorkPays Family included only in overall work participat...          73
         Two-Parent Family included in both the overall ...           5